Group 14: Naveen Makkar, Neel Mansukhani, Tze Hei Tam, Xuecheng Liu

In [1]:
# Imports
import math
import numpy as np
from collections import Counter


# Part 0: Viterbi


Note: Need to upload data to runtime before running.

In [2]:
# Compute counts from train data

# Initialize data structures
tag_given_tag_counts=dict()
word_given_tag_counts=dict()

tag_count = dict({'<s>': 0})

unique_words = set()

with open ("pos_train.txt","r") as infile:
    for line in infile:
        #
        # first tag is the start symbol
        lasttag=["<s>"]

        tag_count['<s>'] += 1
        #
        # split line into word/tag pairs
        #
        for wordtag in line.rstrip().split(" "):
            if wordtag == "":
                continue
            # note that you might have escaped slashes
            # 1\/2/CD means "1/2" "CD"
            # keep 1/2 as 1\/2 
            parts=wordtag.split("/")
            pre_tag=parts.pop()
            word="/".join(parts)

            tag_parts = pre_tag.split("|")
 
            for tag in tag_parts:
              #
              # update counters
              if tag not in word_given_tag_counts:
                  word_given_tag_counts[tag]=Counter()

              if tag not in tag_count:
                tag_count[tag] = 0

              tag_count[tag] += 1

              word_given_tag_counts[tag][word]+=1

            for prev_tag in lasttag:
              if prev_tag not in tag_given_tag_counts:
                  tag_given_tag_counts[prev_tag]=Counter()

            # Update list of words 
            unique_words.add(word)

            for tag in tag_parts:
              for prev_tag in lasttag:
                tag_given_tag_counts[prev_tag][tag]+=1


            lasttag=tag_parts

# Create lists for words and tags, start tag not included
tags = list(word_given_tag_counts.keys())
unique_words = list(unique_words)

The above block of code reads in the training data and processes it. 

In [3]:
# Read in test data

# Initialize data structures
sentences = []
test_tags = []
with open ("pos_test.txt","r") as infile:
    for line in infile:
        #
        # first tag is the start symbol
        lasttag=["<s>"]
        #
        # split line into word/tag pairs
        #

        # Start new word and tag list
        words = []
        sentence_tags = []
        for wordtag in line.rstrip().split(" "):
            if wordtag == "":
                continue
            # note that you might have escaped slashes
            # 1\/2/CD means "1/2" "CD"
            # keep 1/2 as 1\/2 
            parts=wordtag.split("/")
            tag=parts.pop()
            word="/".join(parts)
            #

            tag_parts = tag.split("|")

            # Add word and tag
            words.append(word)
            sentence_tags.append(tag_parts)

            lasttag=tag_parts

        # Add new sentence and its corresponding tags
        sentences.append(words)
        test_tags.append(sentence_tags)

The above block of code reads in the test data and processes it. 

In [4]:
# Compute log probabilities
tag_given_tag_probs = dict()
word_given_tag_probs = dict()

for tag, counts in tag_given_tag_counts.items():
  tag_given_tag_probs[tag] = dict()

  # Compute how many tags led to current tag
  csum = sum(counts.values())
  for element in counts:
    tag_given_tag_probs[tag][element] = math.log(counts[element] / csum)


for tag, counts in word_given_tag_counts.items():
  word_given_tag_probs[tag] = dict()

  # Compute how many words led to current tag 
  csum = sum(counts.values())
  for element in counts:
    word_given_tag_probs[tag][element] = math.log(counts[element] / csum)

The above block of code computes the probabilities given the training data. We compute the observed probability of a tag given the previous tag, and the observed probability of a word given a tag. The log of these probabilities was taken to avoid underflow. 

In [5]:
# Create probability matrices 

# Set value for missing values, because we cannot take log of zero
absent_prob = math.log(.0001)

tag_given_tag_probs_mat = np.ones((len(tags), len(tags))) * absent_prob
word_given_tag_probs_mat = np.ones((len(tags), len(unique_words))) * absent_prob

# Insert probabilities
for i in range(len(tags)):
  for j in range(len(tags)):
    if tags[j] in tag_given_tag_probs[tags[i]]:
      tag_given_tag_probs_mat[i, j] = tag_given_tag_probs[tags[i]][tags[j]]

for j in range(len(tags)):
  for t in range(len(unique_words)):
    if unique_words[t] in word_given_tag_probs[tags[j]]:
      word_given_tag_probs_mat[j, t] = word_given_tag_probs[tags[j]][unique_words[t]]

The above block of code creates matrices to store the transition probabilities and emission probabilities, based on the training data. 

In [6]:
# Create map from words to indices in unique_words list
word_index = dict()

for word in unique_words:
  word_index[word] = unique_words.index(word)

#Probability of most frequent tag
most_frequent_prob = np.log(np.max(list(tag_count.values())) / np.sum(list(tag_count.values())))


The above block of code maps words to an integer ID. The probability of the most frequent tag is calculated as well. If we encounter a word that we don't have a count for, we assume that it belongs to the most frequently encountered tag. 

In [7]:
# Viterbi algorithm
def viterbi(obs, tags, word_index, word_given_tag_probs_mat, tag_given_tag_probs_mat, most_frequent_prob):
  n = len(tags)
  t = len(obs)

  # Compute priors
  priors = np.zeros((n))
  den = t + tag_count['<s>']

  for i in range(n):
    priors[i] = math.log(tag_count[tags[i]] / den)

  # Initialization step
  viterbi = np.zeros((n, t))
  backpointers = np.zeros((n, t))

  for i in range(n):
    if obs[0] in word_index.keys():
      viterbi[i, 0] = priors[i] + word_given_tag_probs_mat[i, word_index[obs[0]]]
    else:
      viterbi[i, 0] = priors[i] + most_frequent_prob

  # Recursive step
  for timestep in range(1, t):
    for state in range(n):
      if obs[timestep] in word_index.keys():
        temp = viterbi[:, timestep - 1] + tag_given_tag_probs_mat[:, state] + word_given_tag_probs_mat[state, word_index[obs[timestep]]]
      else:
        temp = viterbi[:, timestep - 1] + tag_given_tag_probs_mat[:, state] + most_frequent_prob

      viterbi[state, timestep] = np.max(temp) 
      backpointers[state, timestep - 1] = int(np.argmax(temp))

  # Termination
  best_path_prob = np.max(viterbi[:, t - 1])
  best_path_pointer = np.argmax(viterbi[:, t - 1])

  best_path = np.ones(t)
  best_path[-1] = best_path_pointer

  # Backtracking
  for timestep in range(t - 2, -1, -1):
    best_path[timestep] = (int(backpointers[int(best_path[timestep + 1]), timestep]))

  return best_path

The above block of code contains an implementation of the Viterbi algorithm, used to find the most likely tags for a sentence given the sentence itself. 

In [8]:
# Accuracy function
def accuracy(result, expected):
  count = 0
  for i in range(len(result)):
    # Check if predicted tag is one of the acceptable tags
    if int(result[i]) in expected[i]:
      count += 1

  acc = count / len(result)
  return acc

The above block of code calculates the accuracy of results given the expected value

In [9]:
avg_acc = 0

# Run Viterbi algorithm for all sentences in the test dataset
for i in range(len(sentences)):
  path = viterbi(sentences[i], tags, word_index, word_given_tag_probs_mat, tag_given_tag_probs_mat, most_frequent_prob)
  expected = []
  for tag in test_tags[i]:
    tag_pos = []
    for tag_part in tag:
      tag_pos.append(tags.index(tag_part))
    expected.append(tag_pos)

  avg_acc += accuracy(path, expected)

avg_acc /= len(sentences)
print("Average Accuracy: " + str(format(avg_acc, ".3f")))

Average Accuracy: 0.866


The Viterbi algorithm was run for every sentence in the test dataset. Note that if the sentence has a word with more than one possible correct tag, if the predicted tag matches any of the potential tags, it was treated as correct. The average accuracy over every sentence in the test dataset was approximately 86.6%. 

# Extension 1: Forward-Backward


Forward-backward implementation for icecream



In [10]:
# Set up according to lab doc, not working yet

train_sentences = []

# Create map from words to indices in unique_words list
word_index = dict()

for word in unique_words:
  word_index[word] = unique_words.index(word)

with open ("pos_train.txt","r") as infile:
    for line in infile:
        #
        # first tag is the start symbol
        lasttag="<s>"
        #
        # split line into word/tag pairs
        #

        sentence = []
        for wordtag in line.rstrip().split(" "):
            if wordtag == "":
                continue
            # note that you might have escaped slashes
            # 1\/2/CD means "1/2" "CD"
            # keep 1/2 as 1\/2 
            parts=wordtag.split("/")
            tag=parts.pop()
            word="/".join(parts)
            
            sentence.append(word_index[word])

        train_sentences.append(sentence)

The above block of code reads in the training data and processes it for the purposes of the forward-backward algorithm.

In [11]:
def forward_backward_log(obs,prior,A,omega,B):
  obs=np.reshape(obs,(-1))
  T=obs.shape[0]
  S=A.shape[0]
  O=B.shape[1]

  #Expectation
  #Forward
  alpha=np.zeros((T,S),dtype='double')

  alpha[0,:]=np.reshape(prior,(1,-1)) + np.transpose(B[:,obs[0]])
  for i in range(1,T):
    #See Viterbi impl. for what is happening here
    aa=np.reshape(B[:,obs[i]],(1,-1)) + A + np.reshape(alpha[i-1,:],(-1,1))
    #Marginalize over all preceding possibilities
    alpha[i,:]=np.log(np.sum(np.exp(aa),axis=0))

  #Backward
  beta=np.zeros((T,S),dtype='double')
  beta[T-1,:]=np.reshape(omega,(1,-1))
  for i in range(T-2,-1,-1):
    #For each current state, what if it came before each possible succeeding state
    #So matrix with col for current state, row for succ state
    bb=np.reshape(B[:,obs[i+1]],(1,-1)) + A + np.reshape(beta[i+1,:],(1,-1))
    #Marginalize over all possible succeeding states
    beta[i,:]=np.log(np.sum(np.exp(bb),axis=1))
  alphabeta=alpha + beta
  #probability that we passed through state s at time i given the entire sequence
  p_state_at_i=alphabeta - np.reshape(np.log(np.sum(np.exp(alphabeta),axis=1)),(-1,1))
  #Maximization
  s1 = np.sum(np.exp(p_state_at_i),axis=0)
  for i in range(len(s1)):
    if s1[i] == 0:
      s1[i] += 0.0001
  exp_state_instances=np.log(s1)

  #Observations
  Bnew=np.zeros(B.shape,dtype='double')
  for j in range(S):
    for l in range(O):
      s2 = np.sum(np.exp(p_state_at_i[obs==l,j]))
      if s2 == 0:
        s2 += 0.0001
      #print(s2)
      exp_state_obs_instances=np.log(s2)
      Bnew[j,l]=exp_state_obs_instances - exp_state_instances[j]
  #Transitions
  Anew=np.zeros(A.shape,dtype='double')
  for j in range(S):
    for k in range(S):
      #Expected instances where we transitioned from state j to k
      Anew[j,k]=np.log(np.sum(np.exp( alpha[:-1,j] + A[j,k] + beta[1:,k] + B[k,obs[1:]] - np.log(np.sum(np.exp(alphabeta[1:,:]),axis=1)) )))
  Anew=Anew - np.reshape(exp_state_instances,(-1,1))
  priorNew=p_state_at_i[0,:]
  #How likely was state at end vs total instances of that state
  omegaNew=p_state_at_i[-1,:] - exp_state_instances

  return priorNew,Anew,omegaNew,Bnew

The above block of code is an implementation of the forward-backward algorithm, but uses log probabilities to prevent underflow issues. 

In [12]:
A=np.array([[0.8,0.1],[0.1,0.8]],dtype='double') #/(0.8+0.1)
priors=np.array([0.5,0.5],dtype='double')
omega=np.array([0.1,0.1],dtype='double')
B=np.array([[0.7,0.2,0.1],[0.1,0.2,0.7]],dtype='double')
obs=np.array([2,3,3,2,3,2,3,2,2,3,1,3,3,1,1,1,2,1,1,1,3,1,2,1,1,1,2,3,3,2,3,2,2],dtype='int')-1

A = np.log(A)
piors = np.log(priors)
omega = np.log(omega)
B = np.log(B)

for iter in range(10):
  priors,A,omega,B=forward_backward_log(obs,priors,A,omega,B)
  display('-------------','priors',np.exp(priors),'A',np.exp(A),'omega',np.exp(omega),'B',np.exp(B))

'-------------'

'priors'

array([0.12905787, 0.87094213])

'A'

array([[0.87574097, 0.1089602 ],
       [0.09251703, 0.86515797]])

...

array([[0.93372279, 0.06627721],
       [0.07184143, 0.86500846]])

'omega'

array([2.64775193e-14, 6.31501113e-02])

'B'

array([[6.40704102e-01, 1.48087101e-01, 2.11208797e-01],
       [1.57659441e-04, 5.34131667e-01, 4.65710673e-01]])

The above block of code runs the forward-backward algorithm on Eisner's ice cream data for 10 iterations. The results for prior, posterior, transition, and emission probabilities match what was calculated by the Eisner ice cream spreadsheet after 10 iterations of the forward-backward algorithm. 

In [13]:
# Compute log probabilities
tag_given_tag_probs = dict()
word_given_tag_probs = dict()

for tag, counts in tag_given_tag_counts.items():
  tag_given_tag_probs[tag] = dict()

  # Compute how many tags led to current tag
  csum = len(counts)
  for element in counts:
    tag_given_tag_probs[tag][element] = math.log(1 / csum)

for tag, counts in word_given_tag_counts.items():
  word_given_tag_probs[tag] = dict()

  # Compute how many words led to current tag 
  csum = len(counts)
  for element in counts:
    word_given_tag_probs[tag][element] = math.log(1 / csum)


The above block of code calculates the probabilities needed for the forward-backward algorithm. The log of the probabilities was taken to avoid underflow issues. 

In [14]:
# Create probability matrices 

# Set value for missing values
absent_prob = math.log(.0001)

tag_given_tag_probs_mat = np.ones((len(tags), len(tags))) * absent_prob
word_given_tag_probs_mat = np.ones((len(tags), len(unique_words))) * absent_prob

# Insert probabilities
for i in range(len(tags)):
  for j in range(len(tags)):
    if tags[j] in tag_given_tag_probs[tags[i]]:
      tag_given_tag_probs_mat[i, j] = tag_given_tag_probs[tags[i]][tags[j]]

for j in range(len(tags)):
  for t in range(len(unique_words)):
    if unique_words[t] in word_given_tag_probs[tags[j]]:
      word_given_tag_probs_mat[j, t] = word_given_tag_probs[tags[j]][unique_words[t]]


The above block of code creates matrices for the transition and emission probabilities, specifically for use with the forward-backward algorithm. 

In [15]:
#Initialize priors and omega
priors = np.ones(len(tags)) * (1 / len(tags))
omega = np.ones(len(tags)) * (1 / len(tags))

#Take log of priors and omega
priors = np.log(priors)
omega = np.log(omega)

num_iterations = 2
max_length = 10
num_sentences = 5

dev_sentences = []

for sentence in train_sentences:
  if len(sentence) < max_length:
    dev_sentences.append(sentence)

dev_sentences = dev_sentences[:num_sentences]


for i in range(num_iterations):
  print("Iteration: " + str(i))
  for sentence in range(len(dev_sentences)):
    priors, tag_given_tag_probs_mat, omega, word_given_tag_probs_mat = forward_backward_log(dev_sentences[sentence], priors, tag_given_tag_probs_mat, omega, word_given_tag_probs_mat)
  display('-------------','priors',priors,'A',tag_given_tag_probs_mat,'omega',omega,'B',word_given_tag_probs_mat)
  print()



Iteration: 0


'-------------'

'priors'

array([ -4.44039364,  -4.64131393,  -6.1129621 ,  -4.36106187,
        -6.18668733,  -5.02728633,  -4.33937428,  -8.67299699,
        -4.65596366,  -4.3944729 , -36.30361411,  -4.7002295 ,
        -4.6883934 ,  -3.09104157,  -4.55847192,  -4.63421409,
        -3.39981009,  -4.61959765,  -4.70646735,  -4.91905084,
        -2.45224108,  -4.46630857,  -7.56563207,  -4.74597764,
        -3.91181996,  -2.91311758,  -2.17360156,  -4.33838988,
        -4.61230721,  -4.78336425,  -4.88618   ,  -4.63874091,
        -4.03501543,  -4.97468878,  -4.31849089,  -3.06015708,
        -3.04146613,  -5.318646  ,  -1.76806949,  -5.52581949,
        -4.92806698, -10.49780354,  -2.63042743,  -5.93494447,
        -3.24745686])

'A'

...

array([[ -5.15605036,  -9.40237364,  -6.59266612, ...,  -8.53325161,
         -7.91803538, -18.77088417],
       [ -4.4562124 , -10.25107518,  -6.20570756, ..., -14.06064435,
         -2.36082394, -18.57639542],
       [ -5.49748262, -15.03369539,  -7.09892355, ..., -14.09506426,
         -8.22713569, -24.23950594],
       ...,
       [ -2.12945928,  -9.12487828,  -3.66537963, ...,  -7.28806668,
         -8.19147232, -11.72576215],
       [-10.87834682, -10.96660723, -12.43468262, ..., -17.64882663,
         -8.90542659, -22.59533527],
       [ -8.32265087,  -0.30190154,  -1.37434253, ...,  -6.9042715 ,
         -7.30624014, -11.23662528]])

'omega'

array([-1.65471098e+02, -1.63495656e+02, -1.36074674e+02, -1.60598428e+02,
       -1.32305851e+02, -1.66875289e+02, -1.68309371e+02, -2.31580775e+02,
       -1.53151426e+02, -1.65338270e+02, -1.47646118e-10, -1.66525882e+02,
       -1.65810389e+02, -1.91482911e+02, -1.66411946e+02, -1.64131861e+02,
       -2.09920955e+02, -1.64463081e+02, -1.64603136e+02, -1.74756147e+02,
       -2.27113886e+02, -1.65952540e+02, -1.56763444e+02, -1.71290502e+02,
       -2.01754887e+02, -1.84432707e+02, -2.53632446e+02, -1.71002906e+02,
       -1.81226023e+02, -1.58422465e+02, -1.66179622e+02, -1.72305208e+02,
       -1.90390667e+02, -1.35030773e+02, -1.74892509e+02, -1.99354577e+02,
       -1.91775020e+02, -1.62613387e+02, -3.22135016e+02, -1.82960926e+02,
       -1.86011102e+02, -1.98009005e+02, -2.26568167e+02, -1.86469572e+02,
       -2.15408402e+02])

'B'

array([[-6.29823521, -6.29823521, -6.29823521, ..., -6.29823521,
        -6.29823521, -6.29823521],
       [-7.20708374, -7.20708374, -7.20708374, ..., -7.20708374,
        -7.20708374, -7.20708374],
       [-6.45960525, -6.45960525, -6.45960525, ..., -6.45960525,
        -6.45960525, -6.45960525],
       ...,
       [-7.29600264, -7.29600264, -7.29600264, ..., -7.29600264,
        -7.29600264, -7.29600264],
       [-6.41597654, -6.41597654, -6.41597654, ..., -6.41597654,
        -6.41597654, -6.41597654],
       [-7.35819134, -7.35819134, -7.35819134, ..., -7.35819134,
        -7.35819134, -7.35819134]])

The above block of code uses the forward-backward algorithm to find updated probabilities for prior, posterior, transition, and emission regarding sentence tagging. Note that not many iterations were used, and that the training is done using a small subset of the training dataset. This is because the algorithm takes a long time to run. In future trials, this training should be allowed to run for the entire dataset for more iterations to produce more accurate results. 

In [16]:
avg_acc = 0

# Run the Viterbi algorithm on the test set
# Use the updated probabilities from the forward-backward algorithm
for i in range(len(sentences)):
  path = viterbi(sentences[i], tags, word_index, word_given_tag_probs_mat, tag_given_tag_probs_mat, most_frequent_prob)
  expected = []
  for tag in test_tags[i]:
    tag_pos = []
    for tag_part in tag:
      tag_pos.append(tags.index(tag_part))
    expected.append(tag_pos)

  avg_acc += accuracy(path, expected)

avg_acc /= len(sentences)
print("Average Accuracy: " + str(format(avg_acc, ".3f")))

Average Accuracy: 0.052


The Viterbi algorithm was run for every sentence in the test dataset. Note that if the sentence has a word with more than one possible correct tag, if the predicted tag matches any of the potential tags, it was treated as correct. The average accuracy over every sentence in the test dataset was approximately 5.2%. This was significantly worse than the accuracy from the Viterbi algorithm alone in part 0. This is likely because the forward-backward algorithm was trained using few iterations and with a smaller dataset. More robust training would likely produce far better results. 

# Extension 3: Case Insensitive

In [17]:
# Compute counts from train data

# Initialize data structures
tag_given_tag_counts=dict()
word_given_tag_counts=dict()

tag_count = dict({'<s>': 0})

unique_words = set()

with open ("pos_train.txt","r") as infile:
    for line in infile:
        #
        # first tag is the start symbol
        lasttag=["<s>"]

        tag_count['<s>'] += 1
        #
        # split line into word/tag pairs
        #
        for wordtag in line.rstrip().split(" "):
            if wordtag == "":
                continue
            # note that you might have escaped slashes
            # 1\/2/CD means "1/2" "CD"
            # keep 1/2 as 1\/2 
            parts=wordtag.split("/")
            pre_tag=parts.pop()
            word="/".join(parts)

            word = word.lower()
            tag_parts = pre_tag.split("|")
 
            for tag in tag_parts:
              #
              # update counters
              if tag not in word_given_tag_counts:
                  word_given_tag_counts[tag]=Counter()

              if tag not in tag_count:
                tag_count[tag] = 0

              tag_count[tag] += 1

              word_given_tag_counts[tag][word]+=1

            for prev_tag in lasttag:
              if prev_tag not in tag_given_tag_counts:
                  tag_given_tag_counts[prev_tag]=Counter()

            # Update list of words 
            unique_words.add(word)

            for tag in tag_parts:
              for prev_tag in lasttag:
                tag_given_tag_counts[prev_tag][tag]+=1


            lasttag=tag_parts

# Create lists for words and tags, start tag not included
tags = list(word_given_tag_counts.keys())
unique_words = list(unique_words)

The above block of code reads in and processes the training data just as in part 0, but makes every word lower case. 

In [18]:
# Read in test data

# Initialize data structures
sentences = []
test_tags = []
with open ("pos_test.txt","r") as infile:
    for line in infile:
        #
        # first tag is the start symbol
        lasttag=["<s>"]
        #
        # split line into word/tag pairs
        #

        # Start new word and tag list
        words = []
        sentence_tags = []
        for wordtag in line.rstrip().split(" "):
            if wordtag == "":
                continue
            # note that you might have escaped slashes
            # 1\/2/CD means "1/2" "CD"
            # keep 1/2 as 1\/2 
            parts=wordtag.split("/")
            tag=parts.pop()
            word="/".join(parts)
            #
            word = word.lower()
            tag_parts = tag.split("|")

            # Add word and tag
            words.append(word)
            sentence_tags.append(tag_parts)

            lasttag=tag_parts

        # Add new sentence and its corresponding tags
        sentences.append(words)
        test_tags.append(sentence_tags)


The above block of code reads in and processes the test data just as in part 0, but makes every word lower case. 

In [19]:
# Compute log probabilities
tag_given_tag_probs = dict()
word_given_tag_probs = dict()

for tag, counts in tag_given_tag_counts.items():
  tag_given_tag_probs[tag] = dict()

  # Compute how many tags led to current tag
  csum = sum(counts.values())
  for element in counts:
    tag_given_tag_probs[tag][element] = math.log(counts[element] / csum)

for tag, counts in word_given_tag_counts.items():
  word_given_tag_probs[tag] = dict()

  # Compute how many words led to current tag 
  csum = sum(counts.values())
  for element in counts:
    word_given_tag_probs[tag][element] = math.log(counts[element] / csum)

The probabilities are recalculated after the case adjustment was made. 

In [20]:
# Create probability matrices 

# Set value for missing values
absent_prob = math.log(.0001)

tag_given_tag_probs_mat = np.ones((len(tags), len(tags))) * absent_prob
word_given_tag_probs_mat = np.ones((len(tags), len(unique_words))) * absent_prob

# Insert probabilities
for i in range(len(tags)):
  for j in range(len(tags)):
    if tags[j] in tag_given_tag_probs[tags[i]]:
      tag_given_tag_probs_mat[i, j] = tag_given_tag_probs[tags[i]][tags[j]]

for j in range(len(tags)):
  for t in range(len(unique_words)):
    if unique_words[t] in word_given_tag_probs[tags[j]]:
      word_given_tag_probs_mat[j, t] = word_given_tag_probs[tags[j]][unique_words[t]]

The transition and emission probability matrices are updated to reflect the case change, as well. 

In [21]:
# Create map from words to indices in unique_words list
word_index = dict()

for word in unique_words:
  word_index[word] = unique_words.index(word)

most_frequent_prob = np.log(np.max(list(tag_count.values())) / np.sum(list(tag_count.values())))


The above block of code maps words to an integer ID. The probability of the most frequent tag is calculated as well. If we encounter a word that we don't have a count for, we assume that it belongs to the most frequently encountered tag.

In [22]:
avg_acc = 0

# Run Viterbi algorithm on the lower case test dataset
for i in range(len(sentences)):
  path = viterbi(sentences[i], tags, word_index, word_given_tag_probs_mat, tag_given_tag_probs_mat, most_frequent_prob)
  expected = []
  for tag in test_tags[i]:
    tag_pos = []
    for tag_part in tag:
      tag_pos.append(tags.index(tag_part))
    expected.append(tag_pos)

  avg_acc += accuracy(path, expected)

avg_acc /= len(sentences)
print("Average Accuracy: " + str(format(avg_acc, ".3f")))

Average Accuracy: 0.864


The Viterbi algorithm was run for every sentence in the test dataset, but with every word being lower case. Note that if the sentence has a word with more than one possible correct tag, if the predicted tag matches any of the potential tags, it was treated as correct. The average accuracy over every sentence in the test dataset was approximately 86.4%. This was 0.2% lower than the accuracy when words were appropriately capitalized, as calculated in part 0. This indicates that word capitalization does not have a significant impact on sentence tagging. 